In [129]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from scipy import stats

$X= ``Número\ de\ gols\ do\ time\ de\ casa" \sim Poisson(\lambda_x)\\$
$Y= ``Número\ de\ gols\ do\ time\ visitante" \sim Poisson(\lambda_x)$

$$P(X=x)=\frac{\lambda^{x}_{x}e^{-\lambda_x}}{x!}\qquad P(Y=y)=\frac{\lambda^{y}_{y}e^{-\lambda_y}}{y!}\\$$

$$P(X=x, Y=y)=P(X=x)P(Y=y)=\frac{\lambda_{x}^{x}\lambda_{y}^{y}e^{-(\lambda_x+\lambda_y)}}{x!y!}$$

# MODELO 1
xG esperado para a partida

In [130]:
xg_cruzeiro_1410 = (0.14+3.87+1.63+1.56+1.74+0.35+0.66+0.58+0.88+0.97)/10
xg_cuiaba_1410 = (9.72+1.80+0.90+1.18+2.14+0.42+0.91+0.76)/10

In [131]:
#poisson.pmf(k, mu, loc)
cruzeiro_model1 = []
for k in range(0, 11):
  cruzeiro_model1.append(stats.poisson.pmf(k, xg_cruzeiro_1410))

cuiaba_model1 = []
for k in range(0, 11):
  cuiaba_model1.append(stats.poisson.pmf(k, xg_cuiaba_1410))

In [132]:
df_prob1 = pd.DataFrame({'Cuiaba': cuiaba_model1, 'Cruzeiro': cruzeiro_model1})
df_prob1_percent = df_prob1.apply(lambda x: x*100).apply(lambda x: round(x, 2))

In [133]:
df_prob2 = pd.DataFrame()
cuiaba_win_percent = []
cruzeiro_win_percent = []

for i in range(1, len(df_prob1) - 1):
    cuiaba_win_percent.append(df_prob1['Cuiaba'][i + 1] * sum(df_prob1['Cruzeiro'][df_prob1['Cruzeiro'].index < i]))
    cruzeiro_win_percent.append(df_prob1['Cruzeiro'][i + 1] * sum(df_prob1['Cuiaba'][df_prob1['Cuiaba'].index < i]))

df_prob2['Cuiaba Win% if score'] = cuiaba_win_percent
df_prob2['Cruzeiro Win% if score'] = cruzeiro_win_percent
df_prob2.index = pd.RangeIndex(1,10)
df_prob2_percent = df_prob2.apply(lambda x: x*100).apply(lambda x: round(x, 2))

In [134]:
win_cuiaba = sum(df_prob2['Cuiaba Win% if score'])
win_cruzeiro = sum(df_prob2['Cruzeiro Win% if score'])

df_prob3 = pd.DataFrame({'Cuiaba Win%': [win_cuiaba], 'Cruzeiro Win%': [win_cruzeiro]})
df_prob3_percent = df_prob3.apply(lambda x: x*100).apply(lambda x: round(x, 2))

In [149]:
df_1410 = pd.concat([df_prob1, df_prob2, df_prob3], axis=1)
df_1410['empate'] = 1-(df_1410['Cruzeiro Win%'][0] + df_1410['Cuiaba Win%'][0])
df_1410.to_excel('dados/previsoes/previsao1410-cuiabavscruzeiro.xlsx')

# MODELO 2
ATT Rating = (Gols Feitos no campeonato/Partidas Jogadas)/Média de gols da liga\
DEF Rating = (Gols Sofridos no campeonato/Partidas Jogadas)/Média de gols da liga\
EXPECTED GOALS = ATTxDEFxAVGLeague